# 07 - Summarization Pipeline: From Text to Concise Summaries

## Learning Objectives
By the end of this notebook, you will understand:
- Text summarization concepts (extractive vs abstractive)
- Using pre-trained summarization models
- Fine-tuning summarization models
- Evaluation metrics for summarization
- Building production-ready summarization systems
- Handling different text lengths and domains

## Introduction to Text Summarization

Text summarization is the task of creating a concise and coherent summary of a longer document while preserving key information and meaning.

### Types of Summarization:
- **Extractive**: Selects important sentences/phrases from the original text
- **Abstractive**: Generates new text that captures the essence of the original

### Applications:
- News article summarization
- Document summarization for research
- Meeting notes summarization
- Email/message summarization

In [ ]:
# Import necessary libraries
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    pipeline, Trainer, TrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import load_dataset, Dataset, DatasetDict
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from evaluate import load as load_metric
import nltk
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Download NLTK data if needed
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## Part 1: Exploring Pre-trained Summarization Models

In [ ]:
# Load different summarization models
summarization_models = {
    "BART-CNN": "facebook/bart-large-cnn",
    "T5-Small": "t5-small",
    "DistilBART": "sshleifer/distilbart-cnn-12-6",
    "Pegasus-XSum": "google/pegasus-xsum"
}

# Sample article for testing
sample_article = """
The rapid advancement of artificial intelligence (AI) has transformed numerous industries and aspects of daily life. 
Machine learning algorithms, particularly deep neural networks, have achieved remarkable success in tasks ranging 
from image recognition to natural language processing. Companies across various sectors are integrating AI 
technologies to improve efficiency, reduce costs, and enhance user experiences.

However, the widespread adoption of AI also raises significant concerns about job displacement, privacy, and ethical 
implications. Many workers fear that automation will make their jobs obsolete, while others worry about the 
potential misuse of AI for surveillance and manipulation. Experts emphasize the importance of developing AI 
responsibly, with proper regulations and safeguards in place.

Despite these challenges, the potential benefits of AI are substantial. In healthcare, AI-powered diagnostic tools 
can help detect diseases earlier and more accurately than traditional methods. In education, personalized learning 
platforms can adapt to individual student needs, improving learning outcomes. The key is to balance innovation 
with responsibility, ensuring that AI development serves humanity's best interests while minimizing potential risks.
"""

print(f"Sample article ({len(sample_article.split())} words):")
print(sample_article.strip())

# Test different models
print("\n" + "="*60)
print("TESTING DIFFERENT SUMMARIZATION MODELS")
print("="*60)

summaries = {}

for model_name, model_id in summarization_models.items():
    try:
        print(f"\nTesting {model_name} ({model_id})...")
        
        # Create pipeline
        summarizer = pipeline(
            "summarization",
            model=model_id,
            device=0 if torch.cuda.is_available() else -1
        )
        
        # Generate summary
        summary = summarizer(
            sample_article,
            max_length=100,
            min_length=30,
            do_sample=False,
            truncation=True
        )[0]['summary_text']
        
        summaries[model_name] = summary
        
        print(f"Summary ({len(summary.split())} words): {summary}")
        
    except Exception as e:
        print(f"Error with {model_name}: {e}")
        continue

print(f"\nSuccessfully tested {len(summaries)} models")

## Part 2: Loading and Analyzing Summarization Datasets

In [ ]:
# Load CNN/DailyMail dataset (popular for summarization)
print("Loading CNN/DailyMail dataset...")

try:
    # Load a small subset for demonstration
    dataset = load_dataset("cnn_dailymail", "3.0.0", split="train[:1000]")
    
    print(f"Dataset loaded: {len(dataset)} examples")
    print(f"Features: {dataset.features}")
    
    # Examine a sample
    sample = dataset[0]
    print(f"\nSample article (first 300 chars):")
    print(sample['article'][:300] + "...")
    
    print(f"\nSample highlights:")
    print(sample['highlights'])
    
    # Analyze dataset statistics
    article_lengths = [len(example['article'].split()) for example in dataset.select(range(100))]
    highlight_lengths = [len(example['highlights'].split()) for example in dataset.select(range(100))]
    
    print(f"\nDataset statistics (first 100 examples):")
    print(f"Article length - Mean: {np.mean(article_lengths):.1f}, Median: {np.median(article_lengths):.1f}")
    print(f"Highlight length - Mean: {np.mean(highlight_lengths):.1f}, Median: {np.median(highlight_lengths):.1f}")
    print(f"Compression ratio: {np.mean(article_lengths) / np.mean(highlight_lengths):.1f}:1")
    
except Exception as e:
    print(f"Error loading dataset: {e}")
    # Create a synthetic dataset for demonstration
    print("Creating synthetic dataset for demonstration...")
    
    synthetic_data = {
        'article': [
            "The stock market experienced significant volatility today as investors reacted to news about inflation rates. Technology stocks led the decline with major companies seeing drops of 3-5%. Financial analysts suggest this is a temporary correction rather than a long-term trend. The Federal Reserve is expected to make an announcement about interest rates next week.",
            "A new study published in Nature reveals groundbreaking findings about climate change impacts on ocean ecosystems. Researchers found that rising temperatures are affecting marine biodiversity more severely than previously thought. The study analyzed data from over 50 locations worldwide and suggests immediate action is needed to protect marine life.",
            "Local authorities have announced the completion of a major infrastructure project that will improve transportation in the metropolitan area. The new highway system is expected to reduce commute times by 30% and create thousands of jobs. The project took five years to complete and cost $2.3 billion in total."
        ],
        'highlights': [
            "Stock market drops as tech stocks decline 3-5%. Federal Reserve rate announcement expected next week.",
            "New Nature study shows climate change severely impacts marine biodiversity. Immediate action needed to protect ocean life.",
            "New $2.3B highway system completed, expected to reduce commute times by 30% and create jobs."
        ]
    }
    
    dataset = Dataset.from_dict(synthetic_data)
    print(f"Created synthetic dataset with {len(dataset)} examples")

## Part 3: Building a Comprehensive Summarization Pipeline

In [ ]:
class SummarizationPipeline:
    """Comprehensive pipeline for text summarization"""
    
    def __init__(self, model_name="facebook/bart-large-cnn"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.pipeline = None
        self._load_model()
    
    def _load_model(self):
        """Load model and tokenizer"""
        try:
            print(f"Loading summarization model: {self.model_name}")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(self.model_name)
            
            self.pipeline = pipeline(
                "summarization",
                model=self.model,
                tokenizer=self.tokenizer,
                device=0 if torch.cuda.is_available() else -1
            )
            
            print("✓ Model loaded successfully")
            
        except Exception as e:
            print(f"✗ Error loading model: {e}")
            raise
    
    def preprocess_text(self, text):
        """Preprocess text for summarization"""
        # Basic cleaning
        text = text.strip()
        
        # Remove extra whitespace
        text = re.sub(r'\s+', ' ', text)
        
        # Check length constraints
        tokens = self.tokenizer.encode(text)
        max_input_length = self.tokenizer.model_max_length - 100  # Leave room for special tokens
        
        if len(tokens) > max_input_length:
            # Truncate to max length
            truncated_tokens = tokens[:max_input_length]
            text = self.tokenizer.decode(truncated_tokens, skip_special_tokens=True)
            print(f"⚠️  Text truncated to {len(truncated_tokens)} tokens")
        
        return text
    
    def summarize(
        self, 
        text, 
        max_length=150, 
        min_length=50, 
        length_penalty=2.0,
        num_beams=4,
        do_sample=False
    ):
        """Generate summary with customizable parameters"""
        
        # Preprocess
        processed_text = self.preprocess_text(text)
        
        try:
            # Generate summary
            summary = self.pipeline(
                processed_text,
                max_length=max_length,
                min_length=min_length,
                length_penalty=length_penalty,
                num_beams=num_beams,
                do_sample=do_sample,
                truncation=True
            )[0]['summary_text']
            
            return {
                'original_text': text,
                'processed_text': processed_text,
                'summary': summary,
                'original_length': len(text.split()),
                'summary_length': len(summary.split()),
                'compression_ratio': len(text.split()) / len(summary.split())
            }
            
        except Exception as e:
            return {
                'original_text': text,
                'summary': None,
                'error': str(e)
            }
    
    def batch_summarize(self, texts, **kwargs):
        """Summarize multiple texts efficiently"""
        results = []
        
        print(f"Summarizing {len(texts)} texts...")
        
        for i, text in enumerate(texts):
            if i % 10 == 0:
                print(f"Progress: {i}/{len(texts)}")
            
            result = self.summarize(text, **kwargs)
            results.append(result)
        
        return results
    
    def analyze_summaries(self, results):
        """Analyze batch summarization results"""
        successful_results = [r for r in results if 'error' not in r]
        
        if not successful_results:
            print("No successful summarizations to analyze")
            return
        
        compression_ratios = [r['compression_ratio'] for r in successful_results]
        original_lengths = [r['original_length'] for r in successful_results]
        summary_lengths = [r['summary_length'] for r in successful_results]
        
        print(f"\nSummarization Analysis:")
        print(f"  Successful: {len(successful_results)}/{len(results)}")
        print(f"  Average compression ratio: {np.mean(compression_ratios):.2f}:1")
        print(f"  Average original length: {np.mean(original_lengths):.1f} words")
        print(f"  Average summary length: {np.mean(summary_lengths):.1f} words")
        
        return {
            'successful_count': len(successful_results),
            'total_count': len(results),
            'avg_compression_ratio': np.mean(compression_ratios),
            'avg_original_length': np.mean(original_lengths),
            'avg_summary_length': np.mean(summary_lengths)
        }

# Initialize summarization pipeline
summarizer = SummarizationPipeline("sshleifer/distilbart-cnn-12-6")  # Smaller model for demo

# Test with sample article
print("\nTesting summarization pipeline...")
result = summarizer.summarize(sample_article, max_length=80, min_length=30)

print(f"\nSummarization Result:")
print(f"Original length: {result['original_length']} words")
print(f"Summary length: {result['summary_length']} words")
print(f"Compression ratio: {result['compression_ratio']:.2f}:1")
print(f"Summary: {result['summary']}")

## Part 4: Evaluation Metrics for Summarization

In [ ]:
# Implement ROUGE evaluation
try:
    rouge_metric = load_metric("rouge")
    print("✓ ROUGE metric loaded")
except:
    print("⚠️  ROUGE metric not available, implementing basic evaluation")
    rouge_metric = None

def evaluate_summarization(predictions, references):
    """Evaluate summarization quality using ROUGE and other metrics"""
    
    if rouge_metric:
        # ROUGE evaluation
        rouge_scores = rouge_metric.compute(
            predictions=predictions,
            references=references,
            use_stemmer=True
        )
    else:
        # Basic overlap-based evaluation
        rouge_scores = None
    
    # Additional metrics
    compression_ratios = []
    length_differences = []
    
    for pred, ref in zip(predictions, references):
        pred_len = len(pred.split())
        ref_len = len(ref.split())
        
        if pred_len > 0:
            compression_ratios.append(ref_len / pred_len)
        length_differences.append(abs(pred_len - ref_len))
    
    metrics = {
        'avg_compression_ratio': np.mean(compression_ratios) if compression_ratios else 0,
        'avg_length_difference': np.mean(length_differences),
        'rouge_scores': rouge_scores
    }
    
    return metrics

# Test evaluation on dataset samples
print("Testing summarization evaluation...")

# Get a few examples from dataset
test_examples = dataset.select(range(min(5, len(dataset))))
test_articles = [ex['article'] for ex in test_examples]
reference_summaries = [ex['highlights'] for ex in test_examples]

# Generate summaries
print("Generating summaries for evaluation...")
generated_summaries = []

for i, article in enumerate(test_articles):
    result = summarizer.summarize(article, max_length=100, min_length=20)
    if 'error' not in result:
        generated_summaries.append(result['summary'])
    else:
        generated_summaries.append("")
    print(f"Processed {i+1}/{len(test_articles)}")

# Filter out empty summaries
valid_pairs = [(g, r) for g, r in zip(generated_summaries, reference_summaries) if g.strip()]
if valid_pairs:
    valid_generated, valid_references = zip(*valid_pairs)
    
    # Evaluate
    eval_metrics = evaluate_summarization(list(valid_generated), list(valid_references))
    
    print(f"\nEvaluation Results ({len(valid_pairs)} valid pairs):")
    print(f"Average compression ratio: {eval_metrics['avg_compression_ratio']:.2f}:1")
    print(f"Average length difference: {eval_metrics['avg_length_difference']:.1f} words")
    
    if eval_metrics['rouge_scores']:
        rouge = eval_metrics['rouge_scores']
        print(f"ROUGE-1 F1: {rouge['rouge1']:.4f}")
        print(f"ROUGE-2 F1: {rouge['rouge2']:.4f}")
        print(f"ROUGE-L F1: {rouge['rougeL']:.4f}")
    
    # Show example comparison
    print(f"\nExample Comparison:")
    print(f"Original: {test_articles[0][:200]}...")
    print(f"Reference: {reference_summaries[0]}")
    print(f"Generated: {generated_summaries[0]}")
    
else:
    print("No valid summary pairs for evaluation")

## Part 5: Advanced Summarization Techniques

In [ ]:
# Implement different summarization strategies
class AdvancedSummarizer:
    """Advanced summarization with multiple strategies"""
    
    def __init__(self, model_name="sshleifer/distilbart-cnn-12-6"):
        self.base_summarizer = SummarizationPipeline(model_name)
    
    def extractive_summary(self, text, num_sentences=3):
        """Simple extractive summarization using sentence scoring"""
        sentences = nltk.sent_tokenize(text)
        
        if len(sentences) <= num_sentences:
            return ' '.join(sentences)
        
        # Simple scoring based on sentence length and position
        scores = []
        for i, sentence in enumerate(sentences):
            # Score based on length (prefer medium-length sentences)
            length_score = min(len(sentence.split()) / 20, 1.0)
            
            # Score based on position (prefer early sentences)
            position_score = 1.0 - (i / len(sentences))
            
            # Combined score
            total_score = length_score * 0.6 + position_score * 0.4
            scores.append((total_score, sentence))
        
        # Select top sentences
        scores.sort(reverse=True)
        selected_sentences = [sentence for _, sentence in scores[:num_sentences]]
        
        return ' '.join(selected_sentences)
    
    def multi_strategy_summary(self, text, strategies=['abstractive', 'extractive']):
        """Generate summaries using multiple strategies"""
        results = {}
        
        if 'abstractive' in strategies:
            abs_result = self.base_summarizer.summarize(text)
            if 'error' not in abs_result:
                results['abstractive'] = abs_result['summary']
        
        if 'extractive' in strategies:
            results['extractive'] = self.extractive_summary(text)
        
        return results
    
    def adaptive_summary(self, text):
        """Choose best strategy based on text characteristics"""
        words = text.split()
        sentences = nltk.sent_tokenize(text)
        
        avg_sentence_length = len(words) / len(sentences)
        
        # Use extractive for very long texts or texts with long sentences
        if len(words) > 1000 or avg_sentence_length > 30:
            print("Using extractive summarization (long/complex text)")
            return self.extractive_summary(text)
        else:
            print("Using abstractive summarization (standard text)")
            result = self.base_summarizer.summarize(text)
            return result['summary'] if 'error' not in result else self.extractive_summary(text)

# Test advanced summarization
advanced_summarizer = AdvancedSummarizer()

print("Testing advanced summarization strategies...")

# Test with sample article
multi_summaries = advanced_summarizer.multi_strategy_summary(
    sample_article, 
    strategies=['abstractive', 'extractive']
)

print(f"\nMulti-strategy summaries:")
for strategy, summary in multi_summaries.items():
    print(f"\n{strategy.upper()}: {summary}")

# Test adaptive summarization
print(f"\nAdaptive summarization:")
adaptive_summary = advanced_summarizer.adaptive_summary(sample_article)
print(f"Result: {adaptive_summary}")

## Part 6: Domain-Specific Summarization

In [ ]:
# Create domain-specific summarizers
def create_domain_examples():
    """Create examples from different domains"""
    return {
        'news': {
            'text': """
            WASHINGTON - The Federal Reserve announced today a 0.25% increase in interest rates, 
            marking the fourth rate hike this year. Fed Chairman Jerome Powell cited persistent 
            inflation concerns and a robust job market as key factors in the decision. 
            The move was widely anticipated by financial markets, which showed minimal reaction 
            to the announcement. Economists predict this could be the final rate increase of 
            the year as inflation shows signs of moderating. The decision was unanimous among 
            Federal Open Market Committee members.
            """,
            'focus': 'key facts and outcomes'
        },
        'scientific': {
            'text': """
            A new study published in the Journal of Climate Research reveals that Arctic 
            ice is melting at an unprecedented rate. The research team, led by Dr. Sarah 
            Chen from the University of Climate Studies, analyzed satellite data spanning 
            20 years. The findings indicate that ice thickness has decreased by 40% since 
            2003, with accelerated melting observed in the past five years. The study 
            employed advanced machine learning algorithms to process terabytes of satellite 
            imagery and temperature measurements. Researchers warn that this trend could 
            lead to significant sea level rise and disruption of global weather patterns.
            """,
            'focus': 'methodology and findings'
        },
        'business': {
            'text': """
            TechCorp Inc. reported record quarterly earnings of $2.1 billion, exceeding 
            analyst expectations by 15%. The company's cloud services division drove 
            most of the growth, with revenue increasing 45% year-over-year. CEO Michael 
            Roberts attributed the success to strategic acquisitions and expanded 
            international operations. The company also announced a $500 million share 
            buyback program and increased its quarterly dividend by 8%. Looking ahead, 
            TechCorp expects continued growth in AI and cloud computing markets, 
            projecting 25-30% revenue growth for the next quarter.
            """,
            'focus': 'financial performance and outlook'
        }
    }

# Test domain-specific summarization
domain_examples = create_domain_examples()

print("DOMAIN-SPECIFIC SUMMARIZATION")
print("=" * 40)

for domain, example in domain_examples.items():
    print(f"\n{domain.upper()} DOMAIN:")
    print(f"Focus: {example['focus']}")
    
    # Generate summary with different parameters based on domain
    if domain == 'news':
        # News: focus on key facts, shorter summaries
        result = summarizer.summarize(
            example['text'], 
            max_length=60, 
            min_length=25,
            num_beams=6  # More focused beam search
        )
    elif domain == 'scientific':
        # Scientific: preserve methodology and findings
        result = summarizer.summarize(
            example['text'], 
            max_length=80, 
            min_length=40,
            length_penalty=1.5  # Less aggressive length penalty
        )
    elif domain == 'business':
        # Business: include key numbers and outlook
        result = summarizer.summarize(
            example['text'], 
            max_length=70, 
            min_length=35,
            num_beams=4
        )
    
    if 'error' not in result:
        print(f"Summary: {result['summary']}")
        print(f"Compression: {result['compression_ratio']:.1f}:1")
    else:
        print(f"Error: {result['error']}")

## Part 7: Visualization and Analysis

In [ ]:
# Create comprehensive visualization of summarization results
def analyze_and_visualize_summarization():
    """Analyze and visualize summarization performance"""
    
    # Generate summaries for analysis
    analysis_texts = [example['text'] for example in domain_examples.values()]
    analysis_texts.append(sample_article)
    
    # Different length configurations
    length_configs = [
        {'name': 'Short', 'max_length': 50, 'min_length': 20},
        {'name': 'Medium', 'max_length': 80, 'min_length': 40},
        {'name': 'Long', 'max_length': 120, 'min_length': 60}
    ]
    
    results = []
    
    for text in analysis_texts:
        text_results = {'original_length': len(text.split())}
        
        for config in length_configs:
            result = summarizer.summarize(
                text,
                max_length=config['max_length'],
                min_length=config['min_length']
            )
            
            if 'error' not in result:
                text_results[config['name']] = {
                    'length': result['summary_length'],
                    'compression': result['compression_ratio'],
                    'summary': result['summary']
                }
        
        results.append(text_results)
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Summarization Analysis', fontsize=16)
    
    # 1. Summary lengths vs original lengths
    original_lengths = [r['original_length'] for r in results]
    
    for config in length_configs:
        config_lengths = [r[config['name']]['length'] for r in results if config['name'] in r]
        if config_lengths:
            axes[0, 0].scatter(original_lengths[:len(config_lengths)], config_lengths, 
                             label=config['name'], alpha=0.7, s=60)
    
    axes[0, 0].set_xlabel('Original Text Length (words)')
    axes[0, 0].set_ylabel('Summary Length (words)')
    axes[0, 0].set_title('Summary Length vs Original Length')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Compression ratios
    compression_data = []
    config_names = []
    
    for config in length_configs:
        compressions = [r[config['name']]['compression'] for r in results if config['name'] in r]
        if compressions:
            compression_data.append(compressions)
            config_names.append(config['name'])
    
    if compression_data:
        axes[0, 1].boxplot(compression_data, labels=config_names)
        axes[0, 1].set_ylabel('Compression Ratio')
        axes[0, 1].set_title('Compression Ratios by Configuration')
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Word frequency in summaries (for medium length)
    all_summaries = ' '.join([r['Medium']['summary'] for r in results if 'Medium' in r])
    words = re.findall(r'\w+', all_summaries.lower())
    word_freq = Counter(words)
    
    # Remove common stop words and get top words
    stop_words = {'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by', 'a', 'an', 'is', 'are', 'was', 'were'}
    filtered_freq = {word: count for word, count in word_freq.items() if word not in stop_words and len(word) > 2}
    top_words = dict(sorted(filtered_freq.items(), key=lambda x: x[1], reverse=True)[:10])
    
    if top_words:
        axes[1, 0].bar(top_words.keys(), top_words.values())
        axes[1, 0].set_title('Most Common Words in Summaries')
        axes[1, 0].set_xlabel('Words')
        axes[1, 0].set_ylabel('Frequency')
        plt.setp(axes[1, 0].xaxis.get_majorticklabels(), rotation=45)
    
    # 4. Summary length distribution
    all_lengths = []
    for config in length_configs:
        lengths = [r[config['name']]['length'] for r in results if config['name'] in r]
        all_lengths.extend(lengths)
    
    if all_lengths:
        axes[1, 1].hist(all_lengths, bins=10, alpha=0.7, edgecolor='black')
        axes[1, 1].set_xlabel('Summary Length (words)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Distribution of Summary Lengths')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print(f"\nSummarization Statistics:")
    print(f"Texts analyzed: {len(results)}")
    
    for config in length_configs:
        lengths = [r[config['name']]['length'] for r in results if config['name'] in r]
        compressions = [r[config['name']]['compression'] for r in results if config['name'] in r]
        
        if lengths and compressions:
            print(f"\n{config['name']} summaries:")
            print(f"  Average length: {np.mean(lengths):.1f} words")
            print(f"  Average compression: {np.mean(compressions):.1f}:1")
            print(f"  Length range: {min(lengths)}-{max(lengths)} words")

analyze_and_visualize_summarization()

## Part 8: Production Deployment Considerations

In [ ]:
# Production-ready summarization service
class ProductionSummarizer:
    """Production-ready summarization service with monitoring and error handling"""
    
    def __init__(self, model_name="sshleifer/distilbart-cnn-12-6", cache_size=100):
        self.model_name = model_name
        self.cache_size = cache_size
        self.cache = {}  # Simple LRU cache
        self.cache_order = []  # Track cache access order
        
        # Performance metrics
        self.metrics = {
            'requests': 0,
            'cache_hits': 0,
            'errors': 0,
            'total_processing_time': 0
        }
        
        # Load model
        self.summarizer = SummarizationPipeline(model_name)
    
    def _get_cache_key(self, text, params):
        """Generate cache key from text and parameters"""
        import hashlib
        content = f"{text}_{params}"
        return hashlib.md5(content.encode()).hexdigest()[:16]
    
    def _update_cache(self, key, value):
        """Update cache with LRU eviction"""
        if key in self.cache:
            # Move to end (most recently used)
            self.cache_order.remove(key)
            self.cache_order.append(key)
        else:
            # Add new entry
            if len(self.cache) >= self.cache_size:
                # Remove least recently used
                oldest_key = self.cache_order.pop(0)
                del self.cache[oldest_key]
            
            self.cache[key] = value
            self.cache_order.append(key)
    
    def summarize_with_monitoring(self, text, **kwargs):
        """Summarize with performance monitoring and caching"""
        import time
        
        start_time = time.time()
        self.metrics['requests'] += 1
        
        # Check cache
        cache_key = self._get_cache_key(text, str(sorted(kwargs.items())))
        
        if cache_key in self.cache:
            self.metrics['cache_hits'] += 1
            # Move to end of cache order
            self.cache_order.remove(cache_key)
            self.cache_order.append(cache_key)
            
            result = self.cache[cache_key].copy()
            result['cached'] = True
            return result
        
        try:
            # Generate summary
            result = self.summarizer.summarize(text, **kwargs)
            
            if 'error' not in result:
                # Cache successful result
                result['cached'] = False
                self._update_cache(cache_key, result)
            else:
                self.metrics['errors'] += 1
            
            processing_time = time.time() - start_time
            self.metrics['total_processing_time'] += processing_time
            result['processing_time'] = processing_time
            
            return result
            
        except Exception as e:
            self.metrics['errors'] += 1
            return {
                'error': str(e),
                'processing_time': time.time() - start_time
            }
    
    def get_metrics(self):
        """Get performance metrics"""
        metrics = self.metrics.copy()
        
        if metrics['requests'] > 0:
            metrics['cache_hit_rate'] = metrics['cache_hits'] / metrics['requests']
            metrics['error_rate'] = metrics['errors'] / metrics['requests']
            metrics['avg_processing_time'] = metrics['total_processing_time'] / metrics['requests']
        
        metrics['cache_size'] = len(self.cache)
        
        return metrics
    
    def health_check(self):
        """Perform health check"""
        try:
            test_result = self.summarize_with_monitoring(
                "This is a test summary to check if the service is working properly.",
                max_length=30,
                min_length=10
            )
            
            return {
                'status': 'healthy' if 'error' not in test_result else 'unhealthy',
                'test_result': test_result,
                'metrics': self.get_metrics()
            }
        except Exception as e:
            return {
                'status': 'unhealthy',
                'error': str(e)
            }

# Test production summarizer
print("Testing production summarizer...")
prod_summarizer = ProductionSummarizer(cache_size=5)

# Test requests with caching
test_texts = [
    "This is the first test text for summarization.",
    "This is the second test text with different content.",
    "This is the first test text for summarization.",  # Duplicate for cache test
    "Another unique text for testing the summarizer."
]

print("\nProcessing test requests...")
for i, text in enumerate(test_texts):
    result = prod_summarizer.summarize_with_monitoring(text, max_length=50, min_length=20)
    
    if 'error' not in result:
        cache_status = "(cached)" if result.get('cached') else "(new)"
        print(f"Request {i+1}: Success {cache_status} - {result['processing_time']:.3f}s")
        print(f"  Summary: {result['summary']}")
    else:
        print(f"Request {i+1}: Error - {result['error']}")

# Display metrics
print(f"\nPerformance Metrics:")
metrics = prod_summarizer.get_metrics()
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.3f}")
    else:
        print(f"  {key}: {value}")

# Health check
print(f"\nHealth Check:")
health = prod_summarizer.health_check()
print(f"Status: {health['status']}")
if 'error' in health:
    print(f"Error: {health['error']}")

## Summary

In this comprehensive notebook, we explored text summarization from multiple angles:

### 🎯 **What We Accomplished**
1. **Model Comparison**: Tested different pre-trained summarization models
2. **Data Pipeline**: Loaded and analyzed summarization datasets
3. **Custom Pipeline**: Built flexible summarization pipeline with preprocessing
4. **Evaluation Metrics**: Implemented ROUGE and custom evaluation methods
5. **Advanced Techniques**: Explored extractive vs abstractive approaches
6. **Domain Adaptation**: Customized summarization for different domains
7. **Visualization**: Created comprehensive analysis and visualizations
8. **Production System**: Built production-ready service with caching and monitoring

### 🔑 **Key Concepts Mastered**
- **Abstractive vs Extractive**: Understanding different summarization approaches
- **Model Selection**: Choosing appropriate models for different use cases
- **Parameter Tuning**: Optimizing length, beam search, and other parameters
- **Evaluation**: Using ROUGE scores and custom metrics
- **Domain Adaptation**: Customizing approaches for different text types
- **Production Deployment**: Caching, monitoring, and error handling

### 📊 **Best Practices Learned**
- **Preprocessing**: Clean and truncate text appropriately
- **Parameter Selection**: Choose max/min length based on use case
- **Evaluation**: Use multiple metrics to assess quality
- **Caching**: Implement caching for production efficiency
- **Monitoring**: Track performance and error rates
- **Error Handling**: Graceful fallbacks and error recovery

### 🚀 **Next Steps**
- **Notebook 08**: Question Answering systems
- **Notebook 09**: Advanced techniques (LoRA, QLoRA)
- **Notebook 10**: Large Language Models and RLHF

### 💡 **Key Takeaways**
- Different models excel at different types of content
- Parameter tuning significantly affects summary quality
- Production systems need robust error handling and monitoring
- Evaluation is crucial for measuring and improving performance
- Domain-specific customization can significantly improve results

Text summarization is a powerful application that combines multiple NLP techniques. You now have the foundation to build sophisticated summarization systems for any domain or use case!